# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Establish significance level for the duration of the project.
alpha = 0.05

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1\. What test is appropriate for this problem? Does CLT apply?

In [5]:
white = len(data[data.race=='w'])
black = len(data[data.race=='b'])
print(f'Total: {len(data)}; White: {white} ; Black: {black}')

Total: 4870; White: 2435 ; Black: 2435


Since we will be testing the equality two population proportions, a two-sample z-test for proportions is the appropriate test.  The sample sizes are sufficiently large to apply the Central Limit Theorem.

## 2\. What are the null and alternate hypotheses?

Null hypothesis: The proportion of white-sounding resumes that received a call back is equal to the proportion of black-sounding resumes that received a call back.

Alternative hypothesis: The proportion of white-sounding resumes that received a call back is not equal to the proportion of black-sounding resumes that received a call back.

## 3\. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
w_prop = np.sum(w['call']) / len(w)
b_prop = np.sum(b['call']) / len(b)
diff_prop = w_prop - b_prop
print(f'The observed difference is proportions is {diff_prop * 100:.2f}%')

The observed difference is proportions is 3.20%


Bootstrapping:

In [8]:
# Generate bootstrap replicates
np.random.seed(100)
size = 10000
bootstrap_white = np.empty(size)
bootstrap_black = np.empty(size)

for i in range(size):
    bootstrap_sample_white = np.random.choice(w['call'], size=len(w['call']), replace=True)
    bootstrap_sample_black = np.random.choice(b['call'], size=len(b['call']), replace=True)
    bootstrap_white[i] = np.sum(bootstrap_sample_white) / len(bootstrap_sample_white)
    bootstrap_black[i] = np.sum(bootstrap_sample_black) / len(bootstrap_sample_black)
    
# Calculate difference in proportion between the two races.
diff = bootstrap_white - bootstrap_black

In [9]:
# Compute 95% confidence interval using the bootstrap samples.
bootstrap_ci = np.percentile (diff, [2.5, 97.5])
print(f'The 95% confidence interval for difference in proportions between white-sounding names and black-sounding names is ({bootstrap_ci[0]:.3f}, {bootstrap_ci[1]:.3f})')


The 95% confidence interval for difference in proportions between white-sounding names and black-sounding names is (0.017, 0.047)


In [10]:
# Calculate p-value
bootstrap_p_value = np.sum(diff <= 0) / len(diff)
print(f'p-value: {bootstrap_p_value:.2f}; alpha: {alpha}')

p-value: 0.00; alpha: 0.05


The p-value is below the significance level and we can reject the null hypothesis and conclude that there is a difference in the two proportions.

Frequentist:

In [11]:
# Calculate a 95% confidence interval using the frequentist approach.
z = stats.norm.ppf(.975)
margin_of_error = z * np.sqrt((w_prop*(1-w_prop)/len(w)) + (b_prop*(1-b_prop)/len(b)))
ci_upper = diff_prop + margin_of_error
ci_lower = diff_prop - margin_of_error
print(f'A 95% confidence interval for the difference in proportions is ({ci_lower:.3f}, {ci_upper:.3f})')

A 95% confidence interval for the difference in proportions is (0.017, 0.047)


Note that this yields the same result as the bootstrap approach above.

In [12]:
# Perform a z-test for the difference in population proportions.
from statsmodels.stats.proportion import proportions_ztest
z_score, p_value = proportions_ztest([np.sum(w['call']), np.sum(b['call'])], nobs=[len(w), len(b)])

print(f'p-value: {p_value:.2f}; alpha: {alpha}')

p-value: 0.00; alpha: 0.05


Once again, the p-value is below the level of significance and we can reject the null hypothesis that the two proportions are equal.

## 4\. Write a story describing the statistical significance in the context or the original problem.

Based on the tests performed above, we can be 95% confident based on the observed values that the proportion of white-sounding names that get a call back from their application is higher than the proportion of black-sounding names that get a call back.

## 5\. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis does not prove that race/name is the most important factor in callback success.  There is evidence to suggest that it is a factor, but there are over 60 other features of the dataset that could be playing a role in whether or not somebody gets a call back. Future analyses can include these other features to investigate the relationship that they may play in callback success.